In [ ]:
from csi_api import StarSAP2000

sap_model = StarSAP2000('ultimo')


In [2]:
from csi_api import *

In [4]:
cambiar_unidades(sap_model, 12)

Unidades cambiadas a: Ton/m (C).


In [11]:

Lx = 4
Ly = 4
Lz = 4

# Materiales y secciones
h = 0.5
b = 0.3
E: float = 2.1e6
v: float = 0.2

In [6]:
crear_material_concreto(sap_model, E, 'concreto', 210, 2.4, 0.2)

Material de concreto creado exitosamente.


In [7]:
crear_seccion_rectangular(sap_model,"SECTION", "concreto", h, b)

Sección rectangular creada exitosamente.


In [19]:
coords = {
    1: (0, 0, 0),
    2: (Lx, 0, 0),
    3: (Lx, Ly, 0),
    4: (0, Ly, 0),
    5: (0, 0, Lz),
    6: (Lx, 0, Lz),
    7: (Lx, Ly, Lz),
    8: (0, Ly, Lz),
}

crear_nodos(sap_model, list(coords.values()), list(map(str, coords.keys())))


Nodo 1 creado en las coordenadas: (0, 0, 0)
Nodo 2 creado en las coordenadas: (4, 0, 0)
Nodo 3 creado en las coordenadas: (4, 4, 0)
Nodo 4 creado en las coordenadas: (0, 4, 0)
Nodo 5 creado en las coordenadas: (0, 0, 4)
Nodo 6 creado en las coordenadas: (4, 0, 4)
Nodo 7 creado en las coordenadas: (4, 4, 4)
Nodo 8 creado en las coordenadas: (0, 4, 4)


In [21]:
def crear_frame_por_nodos(SapModel, NodeI, NodeJ, PropName, FrameName=""):
    """
    Crea un frame en SAP2000 usando las etiquetas de los nodos inicial y final.

    Parámetros:
    SapModel : objeto de SAP2000
        El objeto del modelo de SAP2000.
    NodeI : str
        Etiqueta del nodo inicial.
    NodeJ : str
        Etiqueta del nodo final.
    PropName : str
        Nombre de la propiedad de la sección (material y dimensiones).
    FrameName : str, opcional
        Nombre del frame (si se deja vacío, SAP2000 asignará un nombre automáticamente).
    """
    FrameName1, ret = SapModel.FrameObj.AddByPoint(
        NodeI,    # Etiqueta del nodo inicial
        NodeJ,    # Etiqueta del nodo final
        FrameName,  # Nombre del frame (vacío para que SAP lo asigne)
        PropName    # Propiedad de la sección
    )

    if ret == 0:
        print(f"Frame creado exitosamente: {FrameName1}")
    else:
        print("Error al crear el frame.")

In [22]:
crear_frame_por_nodos(sap_model, "1", "5", "SECTION")
crear_frame_por_nodos(sap_model, "2", "6", "SECTION")
crear_frame_por_nodos(sap_model, "3", "7", "SECTION")
crear_frame_por_nodos(sap_model, "4", "8", "SECTION")
crear_frame_por_nodos(sap_model, "5", "6", "SECTION")
crear_frame_por_nodos(sap_model, "6", "7", "SECTION")
crear_frame_por_nodos(sap_model, "7", "8", "SECTION")
crear_frame_por_nodos(sap_model, "8", "5", "SECTION")

Frame creado exitosamente: 1
Frame creado exitosamente: 2
Frame creado exitosamente: 3
Frame creado exitosamente: 4
Frame creado exitosamente: 5
Frame creado exitosamente: 6
Frame creado exitosamente: 7
Frame creado exitosamente: 8


In [23]:
def obtener_matriz_transformacion(SapModel, nombre_frame, global_coords=True):
    """
    Obtiene la matriz de transformación de un objeto frame en SAP2000.

    Parámetros:
    SapModel : objeto de SAP2000
        El modelo activo.
    nombre_frame : str
        Nombre del frame del que se quiere obtener la matriz.
    global_coords : bool, opcional
        Si True, devuelve matriz local → global.
        Si False, matriz local → sistema actual.

    Retorna:
    list: Matriz de transformación (lista de 9 floats).
    int: Código de retorno (0 si exitoso).
    """
    matriz = [0.0] * 9  # Inicializar lista de 9 ceros
    ret = SapModel.FrameObj.GetTransformationMatrix(nombre_frame, matriz, global_coords)

    if ret == 0:
        print(f"Matriz de transformación del frame '{nombre_frame}':")
        for i in range(3):
            print(matriz[i*3:(i+1)*3])
    else:
        print(f"❌ Error al obtener la matriz del frame '{nombre_frame}'.")

    return matriz, ret


In [25]:
obtener_matriz_transformacion(sap_model, "1")

❌ Error al obtener la matriz del frame '1'.


([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [(0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0), 0])

In [28]:
agregar_patrones_carga(sap_model, ["CARGA"], [3], [1.0])

Patrón de carga 'CARGA' agregado con éxito.


In [33]:
fx = [100, 0, 0, 0, 0, 0]
fy = [0, 100, 0, 0, 0, 0]
sap_model.PointObj.SetLoadForce("5", "CARGA", fx, Replace=True)
sap_model.PointObj.SetLoadForce("6", "CARGA", fx, Replace=True)
sap_model.PointObj.SetLoadForce("6", "CARGA", fy, Replace=False)
sap_model.PointObj.SetLoadForce("7", "CARGA", fy, Replace=True)

[(0.0, 100.0, 0.0, 0.0, 0.0, 0.0), 0]